In [2]:
#基本函數
import numpy as np
import scipy
import matplotlib.pyplot as plt

#讀取資料
import mne
import pyxdf
from mnelab.io import read_raw

#filter-bank CSP演算法相關
from mne.decoding import CSP
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

#deep-learning model相關
import random
from keras.models import Sequential
from keras.layers import Conv2D, Conv3D, MaxPooling3D, Flatten, Dense, Activation, Dropout
from keras.regularizers import l2
import tensorflow as tf

#繪圖相關
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

#記錄計算時間，儲存模型
import time
from datetime import datetime
import pickle

#固定隨機種子
random.seed(10)

In [3]:
fname = "/Users/alkalisk/Downloads/Recording/tsun_1228.xdf"

#決定要用哪一組steam
#1 means unfilter, 2 uses filter by OPENBCI GUI, 3 is aux
used_eeg = 'obci_eeg2'

streams, header = pyxdf.load_xdf(fname)
for i in streams:
    if i['info']['name'][0] == used_eeg:
        eeg_id = i['info']['stream_id']
        break

raw_ori = read_raw(fname, stream_ids=[eeg_id], fs_new=125)
raw_ori.set_channel_types({used_eeg + '_0':'eeg', used_eeg + '_1':'eeg',
                           used_eeg + '_2':'eeg', used_eeg + '_3':'eeg',
                           used_eeg + '_4':'eeg', used_eeg + '_5':'eeg',
                           used_eeg + '_6':'eeg', used_eeg + '_7':'eeg',
                           used_eeg + '_8':'eeg', used_eeg + '_9':'eeg',
                           used_eeg + '_10':'eeg',used_eeg + '_11':'eeg',
                           used_eeg + '_12':'eeg',used_eeg + '_13':'eeg',
                           used_eeg + '_14':'eeg',used_eeg + '_15':'eeg'})
#for data after 1207
raw_ori.rename_channels({used_eeg + '_0':'0',  used_eeg + '_1':'1',
                         used_eeg + '_2':'2', used_eeg + '_3':'3',
                         used_eeg + '_4':'4',  used_eeg + '_5':'5',
                         used_eeg + '_6':'6',  used_eeg + '_7':'7',
                         used_eeg + '_8':'8',  used_eeg + '_9':'9',
                         used_eeg + '_10':'10', used_eeg + '_11':'11',
                         used_eeg + '_12':'12', used_eeg + '_13':'13',
                         used_eeg + '_14':'14',used_eeg + '_15':'15'})

raw = raw_ori.copy()
events,all_events_id=mne.events_from_annotations(raw,event_id='auto')

Stream 3: Calculated effective sampling rate 94.1184 Hz is different from specified rate 125.0000 Hz.
Stream 3: Calculated effective sampling rate 94.1184 Hz is different from specified rate 125.0000 Hz.


Creating RawArray with float64 data, n_channels=16, n_times=220377
    Range : 0 ... 220376 =      0.000 ...  1763.008 secs
Ready.
Used Annotations descriptions: ['BH', 'EOG-close begin', 'EOG-close end', 'EOG-close trial', 'EOG-open begin', 'EOG-open end', 'EOG-open trial', 'F', 'LH', 'RH', 'SESSION-begin', 'SSVEP_10.0', 'SSVEP_4.3', 'SSVEP_6.0', 'SSVEP_7.6', 'Warm-up trial-SSVEP', 'Warm-up trial-begin', 'Warm-up trial-end', 'Warm-up-begin', 'Warm-up-end', 'Warm-upBH', 'Warm-upF', 'Warm-upLH', 'Warm-upRH', 'calib-begin', 'calib-end', 'idle', 'trial-begin', 'trial-end']


/var/folders/k_/m6hrxyv11xlfgj_0mzzyg_sc0000gp/T/ipykernel_2831/1946301989.py:14: RuntimeWarning: The unit for channel(s) obci_eeg2_0, obci_eeg2_1, obci_eeg2_10, obci_eeg2_11, obci_eeg2_12, obci_eeg2_13, obci_eeg2_14, obci_eeg2_15, obci_eeg2_2, obci_eeg2_3, obci_eeg2_4, obci_eeg2_5, obci_eeg2_6, obci_eeg2_7, obci_eeg2_8, obci_eeg2_9 has changed from NA to V.
  raw_ori.set_channel_types({used_eeg + '_0':'eeg', used_eeg + '_1':'eeg',


In [4]:
#建立所需的event的dict與反轉dict
event_dict = {'BH': all_events_id['BH'], 'F':  all_events_id['F'],
              'LH': all_events_id['LH'], 'RH': all_events_id['RH'],
              'idle': all_events_id['idle'],
              'SSVEP_6': all_events_id['SSVEP_6.0'],
              'SSVEP_4.3': all_events_id['SSVEP_4.3'],
              'SSVEP_7.6': all_events_id['SSVEP_7.6'],
              'SSVEP_10': all_events_id['SSVEP_10.0']}
rev_event_dict = {v: k for k, v in event_dict.items()}

#建立new_events:重新調整label，將目標label重新指定為0~8，其餘非目標event數值全+1000區隔
new_events = events.copy()
new_event_dict = {'BH': 0, 'F':  1, 'LH': 2, 'RH': 3,
                  'idle': 4,
                  'SSVEP_6': 5,
                  'SSVEP_4.3': 6,
                  'SSVEP_7.6': 7,
                  'SSVEP_10': 8}

for event in new_events:
  if event[2] in rev_event_dict:
    event[2] = new_event_dict[rev_event_dict[event[2]]]
  else:
    event[2] += 1000

In [50]:
picks = mne.pick_types(raw.info, eeg=True, stim=False, eog=False, exclude="bads")

epochs = mne.Epochs(
    raw,
    new_events,
    event_id = new_event_dict,
    tmin = 0.3,
    tmax = 2.8,
    picks = picks,
    baseline = None,
    preload = True
)

Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 240 events and 313 original time points ...
0 bad epochs dropped


In [53]:
order_list = ['SSVEP_10',
              'RH','RH','RH','RH','RH',
              'idle','idle',
              'SSVEP_6',
              'F','F','F','F','F',
              'idle','idle',
              'SSVEP_6',
             ]

rs = random.sample(range(0,len(epochs['idle'])), 1)
new_data = epochs['idle'][rs].get_data(copy=True)[0]
mock_raw = mne.io.RawArray(new_data, epochs.info)

for order in order_list:
    rs = random.sample(range(0,len(epochs[order])), 1)
    new_data = epochs[order][rs].get_data(copy=True)[0]
    simulated_raw = mne.io.RawArray(new_data, epochs.info)
    mne.concatenate_raws([mock_raw,simulated_raw])
    

Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=313
    Range : 0 ... 312 =      0.000 ...     2.496 secs
Ready.


In [56]:
mock_raw.save("test_mock_lsl.fif",overwrite=True)

Overwriting existing file.
Writing /Users/alkalisk/PycharmProjects/BrainPainter/demo/P2_20240103/test/test_mock_lsl.fif
Closing /Users/alkalisk/PycharmProjects/BrainPainter/demo/P2_20240103/test/test_mock_lsl.fif
[done]


/var/folders/k_/m6hrxyv11xlfgj_0mzzyg_sc0000gp/T/ipykernel_2831/3037271214.py:1: RuntimeWarning: This filename (/Users/alkalisk/PycharmProjects/BrainPainter/demo/P2_20240103/test/test_mock_lsl.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  mock_raw.save("test_mock_lsl.fif",overwrite=True)


In [57]:
mock_raw

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,125.00 Hz
Highpass,0.00 Hz
Lowpass,62.50 Hz
